In [11]:
import os
import random
from PIL import Image, ImageDraw, ImageFont, ImageFilter
import numpy as np
import matplotlib.pyplot as plt
import cv2

# Spanish historical text (sample 5 pages)
sample_texts = [
    "Don Quijote salió una vez más al campo con su lanza y armadura.",
    "La nobleza del alma se refleja en cada acto del caballero.",
    "En la villa de La Mancha vivía un hidalgo de los de lanza en astillero.",
    "El sabio autor relató hazañas tan grandes como los montes.",
    "Y así comenzó la leyenda del ingenioso hidalgo Don Quijote de la Mancha."
]

# Make output folder
output_dir = "synthetic_renaissance_pages"
os.makedirs(output_dir, exist_ok=True)

# Load a calligraphy or old-style font (fallback to default)
try:
    font = ImageFont.truetype("OldeEnglish.ttf", 28)  # use a historic-looking font if available
except:
    font = ImageFont.truetype("arial.ttf", 28)
def add_noise(img_np, amount=0.02):
    noisy = img_np.copy()
    h, w = noisy.shape
    num_salt = int(amount * h * w)

    # Generate salt (white pixels)
    for _ in range(num_salt):
        y = np.random.randint(0, h)
        x = np.random.randint(0, w)
        noisy[y, x] = 255  # white pixel

    return noisy


def degrade_image(img):
    # Ink bleed
    img = img.filter(ImageFilter.GaussianBlur(radius=1.2))
    
    # Convert to numpy for OpenCV
    img_np = np.array(img.convert("L"))

    # Add smudging using motion blur
    kernel_size = 15
    kernel_motion_blur = np.zeros((kernel_size, kernel_size))
    kernel_motion_blur[int((kernel_size-1)/2), :] = np.ones(kernel_size)
    kernel_motion_blur = kernel_motion_blur / kernel_size
    img_np = cv2.filter2D(img_np, -1, kernel_motion_blur)

    # Add faded effect
    img_np = cv2.addWeighted(img_np, 0.8, 255 * np.ones_like(img_np, dtype=np.uint8), 0.2, 0)

    # Add noise
    img_np = add_noise(img_np, amount=0.01)

    return Image.fromarray(img_np)

# Generate 5 synthetic pages
for i, text in enumerate(sample_texts):
    img = Image.new('RGB', (1000, 200), color='white')
    draw = ImageDraw.Draw(img)
    draw.text((40, 80), text, fill='black', font=font)

    degraded = degrade_image(img)
    file_path = os.path.join(output_dir, f"renaissance_page_{i+1}.png")
    degraded.save(file_path)

    print(f"✅ Saved: {file_path}")

print("🎨 Done! All 5 synthetic degraded images generated.")


✅ Saved: synthetic_renaissance_pages\renaissance_page_1.png
✅ Saved: synthetic_renaissance_pages\renaissance_page_2.png
✅ Saved: synthetic_renaissance_pages\renaissance_page_3.png
✅ Saved: synthetic_renaissance_pages\renaissance_page_4.png
✅ Saved: synthetic_renaissance_pages\renaissance_page_5.png
🎨 Done! All 5 synthetic degraded images generated.


# Evaluation

In [30]:
import lpips
import torch
from PIL import Image
import torchvision.transforms as transforms

# Load images
img1 = Image.open('pdf_images/page_1.png').convert('RGB')
img2 = Image.open('synthetic_renaissance_pages/renaissance_page_1.png').convert('RGB')

# Resize to same dimensions (if needed)
img2 = img2.resize(img1.size)

# Convert to tensors [-1, 1]
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5]*3, std=[0.5]*3)  # scales [0,1] to [-1,1]
])

img1_tensor = transform(img1).unsqueeze(0)
img2_tensor = transform(img2).unsqueeze(0)



In [31]:
loss_fn = lpips.LPIPS(net='alex')  # You can also try 'vgg'
distance = loss_fn(img1_tensor, img2_tensor)
print("LPIPS Distance:", distance.item())


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]
Loading model from: C:\Users\hp\AppData\Roaming\Python\Python312\site-packages\lpips\weights\v0.1\alex.pth
LPIPS Distance: 0.5556963682174683
